# Advent of Code 2021
## [Day 9: Smoke Basin](https://adventofcode.com/2021/day/9) 

#### Load Data

In [1]:
import numpy as np
from collections import deque

In [2]:
import html
html_formatter = get_ipython().display_formatter.formatters['text/html']

def ndarray_to_html(a):
    return "<pre>{}</pre>".format(html.escape(str(a)))

html_formatter.for_type(np.ndarray, ndarray_to_html)
pass

In [3]:
import aocd
input_data = aocd.get_data(day=9, year=2021).split('\n')
height = np.array([list(line) for line in input_data], dtype=int)
height

array([[9, 9, 8, ..., 4, 3, 2],
       [9, 8, 7, ..., 2, 1, 0],
       [9, 9, 9, ..., 3, 2, 3],
       ...,
       [4, 5, 6, ..., 5, 6, 7],
       [3, 5, 6, ..., 6, 8, 8],
       [4, 6, 7, ..., 7, 8, 9]])

In [4]:
test_data = [
    '2199943210',
    '3987894921',
    '9856789892',
    '8767896789',
    '9899965678'
]
test_height = np.array([list(line) for line in test_data], dtype=int)
test_height

array([[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
       [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
       [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
       [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
       [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]])

### Part 1

Pad the height map with `10` values so that each low point has a higher value on each side.

In [5]:
pad_height = np.pad(test_height, 1, constant_values=10)
pad_height

array([[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10,  2,  1,  9,  9,  9,  4,  3,  2,  1,  0, 10],
       [10,  3,  9,  8,  7,  8,  9,  4,  9,  2,  1, 10],
       [10,  9,  8,  5,  6,  7,  8,  9,  8,  9,  2, 10],
       [10,  8,  7,  6,  7,  8,  9,  6,  7,  8,  9, 10],
       [10,  9,  8,  9,  9,  9,  6,  5,  6,  7,  8, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]])

Now, low points (along the horizontal axis) are where we have a negative diff followed by a positive diff.  

In [6]:
np.diff(pad_height, axis=1)

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-8, -1,  8,  0,  0, -5, -1, -1, -1, -1, 10],
       [-7,  6, -1, -1,  1,  1, -5,  5, -7, -1,  9],
       [-1, -1, -3,  1,  1,  1,  1, -1,  1, -7,  8],
       [-2, -1, -1,  1,  1,  1, -3,  1,  1,  1,  1],
       [-1, -1,  1,  0,  0, -3, -1,  1,  1,  1,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

By clipping values to [-1, 1], these points show up as `2` in the second diff.

In [7]:
np.diff(np.diff(pad_height, axis=0).clip(-1, 1), axis=0)

array([[ 0,  2,  2,  0,  0,  0,  2,  2,  2,  2,  2,  0],
       [ 0,  0, -2,  0,  0,  0, -2,  0, -2,  0,  0,  0],
       [ 0, -2,  0,  2,  2,  2,  2, -2,  0, -2,  0,  0],
       [ 0,  2,  2,  0,  0,  0, -2,  0,  0,  0, -2,  0],
       [ 0,  0,  0,  0,  0,  0,  2,  2,  2,  2,  2,  0]])

In [8]:
horiz_low_points = (np.diff(np.diff(pad_height, axis=0).clip(-1, 1), axis=0) == 2)[:,1:-1]
horiz_low_points.astype(int)

array([[1, 1, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]])

Doing the same for `axis=1` gives us the vertical low points.

In [9]:
vert_low_points = (np.diff(np.diff(pad_height, axis=1).clip(-1, 1), axis=1) == 2)[1:-1,:]
vert_low_points.astype(int)

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0]])

Points that are true in both these arrays are the answers:

In [10]:
(horiz_low_points & vert_low_points).astype(int)

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

Find the values at those locations. Note that this uses "boolean mask" array indexing.

In [11]:
test_height[horiz_low_points & vert_low_points]

array([1, 0, 5, 5])

In [12]:
def get_low_points(height):
    pad_height = np.pad(height, 1, constant_values=10)
    
    h_low_mask = (np.diff(np.diff(pad_height, axis=1).clip(-1, 1), axis=1) == 2)[1:-1,:]
    v_low_mask = (np.diff(np.diff(pad_height, axis=0).clip(-1, 1), axis=0) == 2)[:,1:-1]
    
    low_mask = h_low_mask & v_low_mask
    
    return low_mask

def get_risk_levels(height):
    low_points = height[get_low_points(height)]
    risk = low_points + 1
    return risk

get_risk_levels(test_height).sum()

15

#### Part 1 Answer
Find all of the low points on your heightmap.  
**What is the sum of the risk levels of all low points on your heightmap?**

In [13]:
get_risk_levels(height).sum()

475

### Part 2

In [14]:
def get_low_point_coords(height):
    return np.array(np.where(get_low_points(height))).T

get_low_point_coords(test_height)[:5]

array([[0, 1],
       [0, 9],
       [2, 2],
       [4, 6]])

In [15]:
directions = [[-1,  0], [ 1,  0], [ 0, -1], [ 0,  1]]

def flood_basin(height, start):
    visited = np.zeros_like(height, dtype=bool)
    to_visit = deque([np.array(start)])
    while to_visit:
        visiting = to_visit.popleft()
        y, x = visiting
        # print("visiting:", y, x, height.shape)
        if x < 0 or x >= height.shape[1]:
            continue
        if y < 0 or y >= height.shape[0]:
            continue
        if visited[y, x]:
            continue
        if height[y, x] == 9:
            continue
        visited[y, x] = True
        for d in directions:
            to_visit.append(visiting + d)
    return visited

print(flood_basin(test_height, [2, 2]).astype(int))

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0]
 [0 1 1 1 1 1 0 0 0 0]
 [1 1 1 1 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]]


In [16]:
np.count_nonzero(flood_basin(test_height, [2, 2]))

14

In [17]:
def get_basin_sizes(height):
    low_points = np.array(np.where(get_low_points(height))).T
    basin_sizes = []
    for point in low_points:
        basin_size = flood_basin(height, point).sum()
        basin_sizes.append(basin_size)
    return basin_sizes

get_basin_sizes(test_height)

[3, 9, 14, 9]

In [18]:
np.product(sorted(get_basin_sizes(test_height))[-3:])

1134

#### Part 2 Answer
**What do you get if you multiply together the sizes of the three largest basins?**

In [19]:
np.product(sorted(get_basin_sizes(height))[-3:])

1092012